# BAMB! TUTORIAL 3: THE DRIFT-DIFFUSION MODEL

In [ ]:
# If running through Google Colab, run this cell to install pyddm
!pip -q install pyddm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyddm

## 1. Simulating the drift-diffusion model by hand
We're going to simulate the drift-diffusion model in a variety of conditions and study of patterns of reaction times and responses that it produces, to develop some intutions on how the different parameters impact the behavior produced. The idea is to learning how the DDM works, but in practice, it is better to use an optimised library for simulating DDMs in your research.  In parts 2-4, we will learn to use one such library.

### a) Write a function to simulate the DDM

In [ ]:
def run_ddm(drift_rate=.5, noise=.8, bound=1.2, dt=.01, T_dur=4):
    """
    Simulate single run of discrete DDM (stores trajectory of decision variable)

    [X, side, RT] = run_ddm(v, a, z, dt)
    
    Input:
        drift_rate: drift rate
        noise: standard deviation of noise
        bound: the threshold(lower boundary corresponds to -a, upper boundary to a)
        dt: time step for discretized version of dynamics
        T_dur: total runtime, in seconds

    Output:
        X: vector with dynamics of the decision variable until hitting the boundary
        side: +1 if DV hits the upper boundary, -1 if DV hits the lower boundary
        RT: reaction time

    """
    # FILL THIS IN



Once you have written this function, use it simulate 5 trials and plot dynamics of the decision variable and bounds
Use parameter values: drift = 0.5, bound = 1.2, and no non-decision-time.
Use time step dt = 0.001 and a duration of 4 sec.

In [ ]:
bound = 1.2
noise = .8
drift_rate = .5
T_dur = 4
dt = .001

# FILL THIS IN
    
plt.xlabel('Time')
plt.ylabel('Decision variable')
plt.ylim((-bound-.2, bound+.2))
plt.xlim(0, T_dur+.2)
plt.axhline(-bound, c='r')
plt.axhline(bound, c='r')

Repeat with 500 trials.  Add transparency of 0.01 to the lines so that you can still see the density.

In [ ]:
# FILL THIS IN
    
plt.xlabel('Time')
plt.ylabel('Decision variable')
plt.ylim((-bound-.2, bound+.2))
plt.xlim((0, T_dur+.2))
plt.axhline(bound, c='r')
plt.axhline(-bound, c='r')

Notice with transparency that you can see the density of particles which hit a given point.  We will come back to this in Part II.

### b) Plot the RT distribution
Run the DDM 10000 times.  We don't care about the trajectories here, but we do care about the RT and the choice that was made. Plot a histogram, separately for correct and incorrect responses.
Use parameters: drift = 0.5, bound = 1.2, noise = 0.8, dt=.005, T_dur = 4.  Also include a non-decision time of 0.3 s.

In [ ]:
N_trials = 10000
drift_rate = .5
bound = 1.2
noise = .8
dt = .005
T_dur = 4
non_decision_time = .3


# FILL THIS IN 
# Put correct and error response times into the list "correct_rts" and "error_rts" to use the plotting code below.

ax1 = plt.subplot(2,1,1)
h = plt.hist(correct_rts, bins=np.arange(0, T_dur, dt*20))
plt.title("Correct RT distribution")
plt.subplot(2,1,2, sharey=ax1)
plt.hist(error_rts, bins=np.arange(0, T_dur, dt*20))
plt.title("Error RT distribution")
plt.tight_layout()


# 2. Simulating the drift-diffusion model with PyDDM
In practice, we generally want to perform simulations with a simulator instead of by hand.  This is because there are more efficient solutions than simulating individual trajectories.  For instance, many DDMs have closed-form mathematical expressions for the RT distribution.  For those that don't, it is possible to simulate the entire probability distribution of evolving particle density instead of individual particles one by one.

You may find it useful in this section and later sections to consult the [PyDDM documentation](https://pyddm.readthedocs.io/en/stable/), especially the [cookbook](https://pyddm.readthedocs.io/en/stable/cookbook/index.html) and the [quick start guide](https://pyddm.readthedocs.io/en/stable/quickstart.html).

### a) Simulate 10000 trials of the drift-diffusion model with PyDDM and plot the RT distribution
Hint: You will want to create a PyDDM Model object, solve it, and then use the "resample" function on the solution.

In [ ]:
import pyddm

# FILL THIS IN 
# Put correct and error response times into the list "correct_rts" and "error_rts" to use the plotting code below.

ax1 = plt.subplot(2,1,1)
plt.hist(correct_rts, bins=np.arange(0, T_dur, 20*.005))
plt.title("Correct RT distribution")
plt.subplot(2,1,2, sharey=ax1)
plt.hist(error_rts, bins=np.arange(0, T_dur, 20*.005))
plt.title("Error RT distribution")
plt.tight_layout()

### b) Simulate an infinite number of trials of the drift-diffusion model and plot the density
Hint: Every "solution" object contains the full RT distribution as pdf("correct") and pdf("error"), so this should be easier than (a)

In [ ]:
import pyddm

# FILL THIS IN 
# Name your model "m" and put correct and error response times into the
# lists "correct_pdf" and "error_pdf" to use the plotting code below.

ax1 = plt.subplot(2,1,1)
plt.plot(m.t_domain(), correct_pdf)
plt.title("Correct RT distribution")
plt.subplot(2,1,2, sharey=ax1)
plt.plot(m.t_domain(), error_pdf)
plt.title("Error RT distribution")

### c) Use the following code to explore how the RT distribution depends on drift, noise, and bound
Note: The "Fittable" object is a placeholder for a value which has not yet been fit to data.  We will see it again when we fit models to data below.
Hint: Check the "real-time" checkbox once the model gui starts in order to update the plot as you drag the sliders

In [ ]:
import pyddm
from pyddm.plot import model_gui_jupyter

# FILL THIS IN 
# Name your model "m" to use the plotting code below

model_gui_jupyter(m)

# 3. Fitting the drift-diffusion model

### a) Fit a simple DDM for a single subject
We will fit the DDM to a non-human primate subject from [Roitman and Shadlen (2002)](https://www.jneurosci.org/content/22/21/9475) performing the random dot motion task.

[Download data](https://pyddm.readthedocs.io/en/latest/_downloads/bcc1102d5b69c49dac52b49536b87240/roitman_rts.csv)


In [ ]:
import pyddm
import pyddm.plot
import pandas

#First, load the data we wish to use
df_rt = pandas.read_csv("https://raw.githubusercontent.com/mwshinn/PyDDM/master/doc/downloads/roitman_rts.csv")

df_rt = df_rt[df_rt["monkey"] == 1] # Only monkey 1

# FILL THIS IN.
# Create a PyDDM Sample named "sample" using the dataframe above.
# Hint: see https://pyddm.readthedocs.io/en/latest/apidoc/model.html#module-pyddm.sample

sample = pyddm.Sample.from_pandas_dataframe(df_rt, rt_column_name="rt", correct_column_name="correct")

First, let's fit a model that ignores the coherence.  It is probably not going to be a very good model, but it will show us how to fit a model in PyDDM.  We will fit the previous model we built.  This is the same model as before, but replacing the parameters to fit with a pyddm.Fittable placeholder.  We want to fit drift, noise, and non-decision time.  After that, use the model gui to visualize it.

In [ ]:
# FILL THIS IN.
# Create a model named "m" to use the plotting code below

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

Now we perform the fit using the fit_adjust_model function.  Note that, as the name suggests, this changes the model by adjusting the model parameters, so the fitted parameters will be saved within our model.  Use LossRobustLikelihood as the loss function.  (We will see why later.)

In [ ]:
# FILL THIS IN.
# Name the model "m" to be compatible with the display and plotting code below.

Show information about the fit.

In [ ]:
pyddm.display_model(m)

We can also use the model gui again, but this time, to visualise the fit that we just performed.

In [ ]:
pyddm.plot.model_gui_jupyter(model=m, sample=sample)

### b) Fitting a coherence-dependent DDM to a single subject

Notice how behaviour is different depending on the coherence of the random dot motion.  So, we need a Drift class which depends on the coherence condition.  For this, [here is an example from the PyDDM documentation](https://pyddm.readthedocs.io/en/latest/cookbook/driftnoise.html#drift-coh).  The bulk of this is implementing the get_drift function, which returns the drift value at a given time, position in space, and task conditions.  Here, we want the drift to be constant over time and space, but different for different task conditions.  So, we will use the "conditions" argument, and ignore the "t" and "x" arguments.

In [ ]:
class DriftCoherence(pyddm.Drift):
    name = # FILL THIS IN
    required_conditions = # FILL THIS IN - this is from the Roitman-Shadlen dataset
    required_parameters = # FILL THIS IN - we can name this parameter anything we want and use that name in get_drift
    def get_drift(self, t, x, conditions, **kwargs):
        # FILL THIS IN

Now, we can construct the final model and fit it to data.  Create a Model object which uses your new Drift class and visualise it with your sample using pyddm.plot.model_gui_jupyter.

In [ ]:
# FILL THIS IN
# Name the model "m" to be compatible with the plotting code below

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

Now, fit your model to data using fit_adjust_model.  Use RobustLikelihood as a loss function.

In [ ]:
# FILL THIS IN

### c) (optional) Plot the psychometric and chronometric functions
The psychometric function shows the coherence/evidence on the x axis and the probability of a correct response on the y axis.  Likewise, the chronometric function shows the coherence/evidence on the x axis and the mean RT of correct responses on the y axis.

Hint: PyDDM model Solutions (the output of m.solve()) have a prob("correct") and prob("error") methods, as well as mean_decision_time() function.

Hint 2: PyDDM Samples have these methods too!  You might also want to use the "subset" method.

In [ ]:
# FILL THIS IN

### d) Adding an explicit model of lapse rate
What happens if the subject responds during the non-decision time?  Well, in theory, the model should give a likelihood of zero, and hence, a negative log likelihood of infinity.  (More generally, if there is even one "outlier" response at a time when the model predicts there should be none, this will have a large effect on the model.)  But when you look at our data, there are indeed a few responses during the non-decision time.  So why is the likelihood finite?

It is finite because we have been cheating a bit.  Notice how when we fit the model previously, we used "Robust Likelihood".  What robust likelihood does is add a small constant offset to the likelihood of each point before taking the logarithm.  Is this best practice?  No, but lots of people still do it.  So let's properly model the lapse trials in addition to the actual trials.

We do this using a mixture model.  We assume that X% of trials are generated by the DDM, and (100-X)% of trials are generated by some other process, for example, an evidence-independent probability distribution.  The two best choices to use for this are a uniform distribution, which assumes lapse trials are equally likely at any point in the trial, or an exponential distribution, which assumes lapse trials come from a poisson distribution (a flat hazard).

We implement mixture models in PyDDM using an overlay.  However, since we are already using an overlay for non-decision time, we need to chain overlays together using pyddm.OverlayChain.  This can be used as follows:

```
overlay=pyddm.OverlayChain(overlays=[overlay1, overlay2]),
```

where "overlay1" and "overlay2" are overlays.  See [the PyDDM documentation](https://pyddm.readthedocs.io/en/latest/apidoc/dependences.html#module-ddm.models.overlay) for a list of all overlays included by default.  Once you are using a mixture model to account for lapse/outlier responses, you can change the loss function from "LossRobustLikelihood" to "LossLikelihood".

Below, modify our model to use an error distribution with a uniform distribution.  Use a fittable mixture ratio.

In [ ]:
# FILL THIS IN
# Name the model "m" to be compatible with the plotting code below.

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

# 4. Generalized drift-diffusion models
Generalized drift-diffusion models (GDDMs) allow going beyond the standard model parameters of the DDM.  Instead of drift, noise, and bound being fixed values, GDDMs allow them to be functions which may vary across time.  For example, this allows modelling tasks which have evidence that changes over time.  It also allows these to have complex, non-linear relationships with any number of task conditions and use any number of parameters.  For example, it is possible to model multisensory integration, with different streams of evidence contributing non-linearly to drift rate.  Furthermore, it also allows integration to be leaky (i.e. forgetting) or unstable (i.e. biasing early evidence), as well as representing an urgency signal (e.g. bounds which collapse over time).  There is evidence that these model properties are useful for modelling RTs in overtrained human or animal subjects.

All of these exercises are optional and do not depend on each other - feel free to skip around and do those which are of greatest interest.

### a) Collapsing boundaries
Sometimes, especially in the case of overtrained animals, more evidence may be needed to make a decision earlier in the trial compared to later in the trial.  Construct, visualise, and fit a model with linearly collapsing boundaries to the data from (3).  It might take a bit longer to fit.

In [ ]:
# FILL THIS IN
# Name the model "m" to be compatible with the plotting code below.

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

In [ ]:
pyddm.fit_adjust_model(model=m, sample=sample, lossfunction=pyddm.LossRobustLikelihood, verbose=False)

### b) Leaky integration
Leaky integration occurs when the decision variable is constantly being pushed back to zero (a stable fixed point at zero).  This models forgetting, or alternatively, prioritising more recent evidence.  This is implemented in the model by making the drift rate depend on the position of the particle at any given time.  Construct and visualise a leaky integration model by modifying the DriftCoherence model above.  You do not need to fit it to data, since this may take a few minutes (and can be done the same way as all the above examples).  Note that leak can be negative: this is also called "unstable integration".

In [ ]:
class DriftCoherenceLeaky(pyddm.Drift):
    name = # FILL THIS IN
    required_conditions = # FILL THIS IN
    required_parameters = # FILL THIS IN
    def get_drift(self, t, x, conditions, **kwargs):
        # FILL THIS IN
    
m = pyddm.Model(drift=DriftCoherenceLeaky(drift=pyddm.Fittable(minval=-20, maxval=20), leak=pyddm.Fittable(minval=-2, maxval=2)),
                noise=pyddm.NoiseConstant(noise=pyddm.Fittable(minval=.1, maxval=2)),
                overlay=pyddm.OverlayNonDecision(nondectime=pyddm.Fittable(minval=0, maxval=.5)))

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

### c) Side bias
In our dataset, we also have information about which side the monkey chose to get the correct answer ("trgchoice" in the Roitman dataset, which has values "1" or "2").  Let's use a GDDM to implement a side bias.

There are two common ways to implement a side bias.  The first is to assume that the biased side causes a constant offset bias on the drift rate.  So, in the 0% coherence condition, the drift rate will be towards the biased side.  Likewise, in a strong evidence condition, the drift rate will be stronger if it is on the same side as the bias.  This can be implemented by adding "trgchoice" as a "required_condition" to the drift rate and a parameter "side_bias" to describe the magnitude of the bias.  Then, when computing the drift function, add "side_bias" to the result if it is on the preferred side, and otherwise, subtract "side_bias". 

In [ ]:
class DriftCoherenceSideBias(pyddm.Drift):
    name = # FILL THIS IN
    required_conditions = # FILL THIS IN - Remember, we need two conditions now
    required_parameters = # FILL THIS IN  - Remember, we need two parameters now
    def get_drift(self, t, x, conditions, **kwargs):
        # FILL THIS IN

# FILL THIS IN
# Create a model and name it "m" to be compatible with the plotting code below

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

The other way is to assume that there is an offset in the starting position: instead of starting at zero, we start at some other point.  To do this, we need to implement a new InitialConditions, which takes the side as a condition and a parameter x0 of the magnitude of the bias.  Then, it returns a probability distribution where all the density is in a single point: the point x0 for one side, or -x0 for the other side.

See the [PyDDM documentation](https://pyddm.readthedocs.io/en/stable/cookbook/initialconditions.html#biased-initial-conditions) for an example.

In [ ]:
class ICPointSideBias(pyddm.InitialCondition):
    name = # FILL TIHS IN
    required_parameters = # FILL THIS IN
    required_conditions = # FILL THIS IN
    def get_IC(self, t, x, dx, conditions):
        # FILL THIS IN

# FILL THIS IN
# Create a model and name it "m" to be compatible with the plotting code below

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

### d) Distributions of starting positions and non-decision time
Suppose that, instead of starting at the position 0, the starting position of the integrator was pulled from a uniform distribution (where the size is a fittable parameter), and the non-decision time is pulled from a gamma distribution with fittable rate and shape parameters.

Hint: PyDDM has built-in an OverlayNonDecisionGamma, as well an ICRange which may be helpful.

In [ ]:
# FILL THIS IN
# Create a model and name it "m" to be compatible with the plotting code below

pyddm.plot.model_gui_jupyter(model=m, sample=sample)

### e) Evidence which changes over time
Evidence is not always the same over time.  For instance, many tasks present discrete pulses of evidence.  Others may have evidence which is constantly fluctuating (e.g. changes in motion energy).

To model this in PyDDM, we need to create a custom Drift object, as we did above.  But this time, the get_drift function should use the "t" function argument.

Model a situation where we have two pulses of evidence, the first lasting from time 0.3 s to 0.6 s, and the second from 1.0 to 1.2 s.  Let the magnitude of each be given by the conditions "pulse1" and "pulse2".  Play with this model in the model_gui.

In [ ]:
import pyddm
import pyddm.plot
import numpy as np
class DriftPulses(pyddm.Drift):
    name = # FILL THIS IN
    required_parameters = # FILL THIS IN
    required_conditions = # FILL THIS IN
    def get_drift(self, t, x, conditions, **_):
        # FILL THIS IN

# FILL THIS IN
# Define a model using this Drift object and call it "m" to be compatible with the plotting code below.

pyddm.plot.model_gui_jupyter(m, conditions={"pulse1": [0, .2, .4, .6], "pulse2": [0, .2, .4, .6]})